# Balanceo de Clases

En problemas de clasificación, especialmente aquellos del mundo real como detección de fraudes o diagnóstico de enfermedades, es común enfrentarse a conjuntos de datos con clases desbalanceadas, donde ua clase está significativamente menos representada que la(s) otra(s).

Este desbalance puede causar que los modelos aprendan a favorecer la clase mayoritaria, obteniendo métricas aparentemente buenas pero con un desempeño deficiente en la clase minoritaria.

Para mitigar este problema, se aplican técnicas de balanceo de clases, como el sobremuestreo, submuestreo o el uso de algoritmos y métricas sensibles al desbalance.

Partamos cargando todas las librerías que utilizaremos.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling  import RandomUnderSampler
from imblearn.combine         import SMOTETomek
from imblearn.pipeline        import Pipeline as ImbPipeline
from sklearn.linear_model     import LogisticRegression
from sklearn.metrics          import classification_report, roc_auc_score

Carguemos las métricas para evaluar las predicción.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

Carguemos el dataset a utilizar.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/titanic-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/titanic-dataset


In [ ]:
df = pd.read_csv(path + "/Titanic-Dataset.csv")

Separemos las variables entre numéricas y categóricas.

In [ ]:
num = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
cat = ['Sex', 'Embarked']

Tomemos las siguientes consideraciones.

In [ ]:
# Eliminamos la columna Cabin
df.drop('Cabin', axis=1, inplace=True)

# Cambiamos los nulos por la mediana en Age, debido a que son valores enteros.
df['Age'].fillna(df['Age'].median(), inplace=True)

# Cambiamos los nulos por la moda en Embarked, debido a que son valores
# categoricos.
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

<ipython-input-7-3c16328db967>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-7-3c16328db967>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [ ]:
X = df[num + cat]
y = df['Survived']

Dividamos los datos entre entrenamiento y validación.

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Realicemo el preprocesamiento de los datos. Recordemos que si hay muchos outliers ocupamos RobustScaler en vez de StandardScaler.

In [ ]:
prep = ColumnTransformer([
    ('num', RobustScaler(), num),
    ('cat', OneHotEncoder(), cat)
])

Partamos realizando el balanceo implementado en la Logistic Regression.

In [ ]:
logistic_reg_balanced = Pipeline([
    ('prep', prep),
    ('logi_reg', LogisticRegression(random_state=42, # Semilla
                                    max_iter=1000, # Maximo de iteraciones
                                    solver='liblinear', # Metodo de optimizacion
                                    penalty = 'l1', # Funcion de penalizacion
                                    C = 0.1, # Parametro de regularizacion
                                    class_weight='balanced' # Balanceo de clases
                                    ))
])

Entrenamiento.

In [ ]:
logistic_reg_balanced.fit(X_tr, y_tr)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logi_reg',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=1000, penalty='l1',
                                    random_state=42, solver='liblinear'))])

Realicemos la predicción del modelo y evaluemos el modelo a través de las métricas

In [ ]:
pred_logis_balanced = logistic_reg_balanced.predict(X_te)

print('Acurracy :',accuracy_score(y_te, pred_logis_balanced))
print('Precision :',precision_score(y_te, pred_logis_balanced))
print('Recall :',recall_score(y_te, pred_logis_balanced))
print('f1_score',f1_score(y_te, pred_logis_balanced))
print('roc_auc_score',roc_auc_score(y_te, pred_logis_balanced))

Acurracy : 0.7988826815642458
Precision : 0.72
Recall : 0.782608695652174
f1_score 0.75
roc_auc_score 0.7958498023715415


El siguiente balanceo lo que hace es interpolar datos sinteticos entre las filas balanceando el número de clases.

In [ ]:
smote = SMOTETomek(random_state=42)

Aquí, antes de entrar a la regresión logistica se agregan filas a los datos a través de smote, balanceando el número de clases.

In [ ]:
pipe_smote = ImbPipeline([
    ('prep', prep),
    ('smote', smote),
    ('logi_reg_smote', LogisticRegression(
        random_state=42,
        max_iter=1000,
        solver='liblinear',
        penalty='l1',
        C=0.1,
        class_weight='balanced'
    ))
])

Entrenamiento.

In [ ]:
pipe_smote.fit(X_tr, y_tr)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('smote', SMOTETomek(random_state=42)),
                ('logi_reg_smote',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=1000, penalty='l1',
                                    random_state=42, solver='liblinear'))])

Ahora, veremos si esto mejoró los resultados vistos sin smote.

In [ ]:
# Predicción
y_pred_smote = pipe_smote.predict(X_te)

# Métricas
print('Accuracy: ', accuracy_score(y_te, y_pred_smote))
print('Precision: ',precision_score(y_te, y_pred_smote))
print('Recall: ', recall_score(y_te, y_pred_smote))
print('f1_score: ', f1_score(y_te, y_pred_smote))
print('roc_auc_score: ', roc_auc_score(y_te, y_pred_smote))

Accuracy:  0.8044692737430168
Precision:  0.7297297297297297
Recall:  0.782608695652174
f1_score:  0.7552447552447552
roc_auc_score:  0.800395256916996


Como podemos ver mejoramos en las métricas por lo que el modelo mejoró con este nuevo balanceo.